In [1]:
from CosinorPy import file_parser, cosinor, cosinor1, cosinor_nonlin
import numpy as np
import pandas as pd
import os

In [2]:
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
rs = RandomState(MT19937(SeedSequence(123456789)))

In [3]:
import importlib
importlib.reload(file_parser)
importlib.reload(cosinor)
importlib.reload(cosinor1)
importlib.reload(cosinor_nonlin)

<module 'CosinorPy.cosinor_nonlin' from 'C:\\Users\\miham\\OneDrive - Univerza v Ljubljani\\Research\\CosinorPy\\code\\CosinorPy\\cosinor_nonlin.py'>

## General settings

In [4]:
# number of samples per a configuration
N = 100

noise_simple = 0
time_step = 2
min_time = 0
max_time = 72
replicates = 3

plot = False

#csv_folder = "evaluation"
external_file=False


AMPLITUDE = [1]
NOISE = [0.1, 0.2, 0.5, 1]
LIN_COMP = [0]
AMPLIFICATION = [0, 0.05, -0.05]
N_COMPONENTS = [1,3]

In [5]:
columns = ['test', 'amplitude', 'acrophase', 'amplification', 'lin_comp', 'p', 'n_components']

results_columns = ['gen', 'test(noise)', 'test(lin_comp)', 'test(amplification)', 'test(n_components)','amplitude0', 'acrophase0', 'amplification0', 'lin_comp0']

results_columns += columns

### Configurations (1 vs. 3 comp)

In [6]:
def run_tests():

    file_name = f"ampl_{amplitude}_noise_{noise}_lin_{lin_comp}_amplif_{amplification}_n_comps_{n_components}.csv"

    if external_file:
        df = pd.read_csv(os.path.join(csv_folder,file_name))
        df_test = pr.read_csv(os.path.join(csv_folder,"params_"+file_name))
    else:  
        """
        phases = [2*np.pi*np.random.random(), 2*np.pi*np.random.random(), 2*np.pi*np.random.random()]
        amplitudes = [amplitude,0.5*np.random.random(),0.5*np.random.random()]
        baseline = 1
        df, df_test = file_parser.generate_test_data_group(N=N, name_prefix="test",
                                                  n_components = n_components, 
                                                  baseline = baseline, 
                                                  phase = phases, 
                                                  amplitudes = amplitudes, 
                                                  noise=noise,                                      
                                                  noise_simple = noise_simple, 
                                                  amplification = amplification,
                                                  lin_comp = lin_comp,
                                                  time_step=time_step, 
                                                  min_time=min_time, 
                                                  max_time = max_time,
                                                  replicates= replicates,
                                                  characterize_data = True)
        """                
        baseline = 1
        df, df_test = file_parser.generate_test_data_group_random(N=N, name_prefix="test",
                                                  n_components = n_components, 
                                                  baseline = baseline, 
                                                  amplitude = amplitude, 
                                                  noise=noise,                                      
                                                  noise_simple = noise_simple, 
                                                  amplification = amplification,
                                                  lin_comp = lin_comp,
                                                  time_step=time_step, 
                                                  min_time=min_time, 
                                                  max_time = max_time,
                                                  replicates= replicates,
                                                  characterize_data = True)

        df.to_csv(os.path.join(csv_folder,file_name), index=False)
        df_test.to_csv(os.path.join(csv_folder,"params_"+file_name), index=False)
    
    df_test = df_test[['test', 'amplitude', 'acrophase', 'amplification', 'lin_comp']]
    df_test.columns = ['test', 'amplitude0', 'acrophase0', 'amplification0', 'lin_comp0']
    
    
    df_results_test = pd.DataFrame(columns=columns, dtype=float)

    #print('1-comp')
    df_results = cosinor_nonlin.fit_generalized_cosinor_n_comp_group(df, n_components = 1, period = 24, plot=plot, folder="", amp_comp = False, lin_comp = False) # , folder="test_f")    
    df_results = df_results[columns]    
    df_results['gen'] = 0    
    df_results = df_results.merge(df_test, how="left", on="test")    
    df_results_test = df_results_test.append(df_results, ignore_index=True, sort=False)
        
    #print('3-comp')    
    df_results = cosinor_nonlin.fit_generalized_cosinor_n_comp_group(df, n_components = 3, period = 24, plot=plot, folder="", amp_comp = False, lin_comp = False) # , folder="test_f")    
    df_results = df_results[columns]
    df_results['gen'] = 0        
    df_results = df_results.merge(df_test, how="left", on="test")
    df_results_test = df_results_test.append(df_results, ignore_index=True, sort=False)

    #print('Gen 1-comp')    
    df_results = cosinor_nonlin.fit_generalized_cosinor_n_comp_group(df, n_components = 1, period = 24, plot=plot, folder="", lin_comp=False) # , folder="test_f")
    df_results = df_results[columns]    
    df_results['gen'] = 1    
    df_results = df_results.merge(df_test, how="left", on="test")
    df_results_test = df_results_test.append(df_results, ignore_index=True, sort=False)
    
    #print('Gen 3-comp')
    df_results = cosinor_nonlin.fit_generalized_cosinor_n_comp_group(df, n_components = 3, period = 24, plot=plot, folder="", lin_comp=False) # , folder="test_f")    
    df_results = df_results[columns]    
    df_results['gen'] = 1    
    df_results = df_results.merge(df_test, how="left", on="test")
    df_results_test = df_results_test.append(df_results, ignore_index=True, sort=False)
    
    df_results_test['test(amplitude)'] = amplitude
    df_results_test['test(noise)'] = noise
    df_results_test['test(lin_comp)'] = lin_comp
    df_results_test['test(amplification)'] = amplification
    df_results_test['test(n_components)'] = n_components
    
    return df_results_test


In [7]:
df_results_all = pd.DataFrame(columns=results_columns, dtype=float)

for noise in NOISE:   
    for n_components in N_COMPONENTS:        
        for amplitude in AMPLITUDE:            
            for lin_comp in LIN_COMP:                
                for amplification in AMPLIFICATION:
                    print("***")
                    print("noise:", noise)
                    print("n_components:", n_components)
                    print("amplitude:", amplitude)
                    print("lin_comp:", lin_comp)
                    print("amplification:", amplification)
                    print("***")
                    df_results = run_tests()
                    df_results = df_results[results_columns]
                    df_results_all = df_results_all.append(df_results, ignore_index=True, sort=False)
                                        

***
noise: 0.1
n_components: 1
amplitude: 1
lin_comp: 0
amplification: 0
***
***
noise: 0.1
n_components: 1
amplitude: 1
lin_comp: 0
amplification: 0.05
***
***
noise: 0.1
n_components: 1
amplitude: 1
lin_comp: 0
amplification: -0.05
***
***
noise: 0.1
n_components: 3
amplitude: 1
lin_comp: 0
amplification: 0
***
***
noise: 0.1
n_components: 3
amplitude: 1
lin_comp: 0
amplification: 0.05
***
***
noise: 0.1
n_components: 3
amplitude: 1
lin_comp: 0
amplification: -0.05
***
***
noise: 0.2
n_components: 1
amplitude: 1
lin_comp: 0
amplification: 0
***
***
noise: 0.2
n_components: 1
amplitude: 1
lin_comp: 0
amplification: 0.05
***
***
noise: 0.2
n_components: 1
amplitude: 1
lin_comp: 0
amplification: -0.05
***
***
noise: 0.2
n_components: 3
amplitude: 1
lin_comp: 0
amplification: 0
***
***
noise: 0.2
n_components: 3
amplitude: 1
lin_comp: 0
amplification: 0.05
***
***
noise: 0.2
n_components: 3
amplitude: 1
lin_comp: 0
amplification: -0.05
***
***
noise: 0.5
n_components: 1
amplitude: 1
lin_

In [8]:
df_results_all

,gen,test(noise),test(lin_comp),test(amplification),test(n_components),amplitude0,acrophase0,amplification0,lin_comp0,test,amplitude,acrophase,amplification,lin_comp,p,n_components
0,0.0,0.1,0.0,0.00,1.0,0.999998,1.490606,0.00,0.0,test_0,1.011401,1.490606,-9.999996e-11,9.999999e-11,1.110223e-16,1.0
1,0.0,0.1,0.0,0.00,1.0,0.999999,1.113237,0.00,0.0,test_1,0.976211,1.113237,9.999990e-11,-9.999995e-11,1.110223e-16,1.0
2,0.0,0.1,0.0,0.00,1.0,0.999999,3.075553,0.00,0.0,test_2,1.029291,3.062974,-9.999999e-11,-9.999999e-11,1.110223e-16,1.0
3,0.0,0.1,0.0,0.00,1.0,0.999998,-2.654158,0.00,0.0,test_3,0.979002,-2.666737,-1.000000e-10,-9.999998e-11,1.110223e-16,1.0
4,0.0,0.1,0.0,0.00,1.0,0.999999,3.075553,0.00,0.0,test_4,1.028491,3.075553,9.999999e-11,9.999995e-11,1.110223e-16,1.0
5,0.0,0.1,0.0,0.00,1.0,0.999998,-0.666684,0.00,0.0,test_5,1.020659,-0.679263,-9.999999e-11,-9.999998e-11,1.110223e-16,1.0
6,0.0,0.1,0.0,0.00,1.0,0.999999,-3.031527,0.00,0.0,test_6,0.947357,-3.006369,-9.999998e-11,-9.999997e-11,1.110223e-16,1.0
7,0.0,0.1,0.0,0.00,1.0,0.999999,-1.308211,0.00,0.0,test_7,0.994545,-1.320790,9.999995e-11,9.999957e-11,1.110223e-16,1.0
8,0.0,0.1,0.0,0.00,1.0,0.999998,-1.358527,0.00,0.0,test_8,1.010992,-1.345948,-9.999998e-11,-9.999999e-11,1.110223e-16,1.0
9,0.0,0.1,0.0,0.00,1.0,0.999998,1.138395,0.00,0.0,test_9,1.025938,1.125816,9.999998e-11,-9.999998e-11,1.110223e-16,1.0


In [9]:
#df_results_all.to_csv(os.path.join(csv_folder,f'evalutation_results_{N}.csv'), index=False)
df_results_all.to_csv(os.path.join(os.path.join("supp_tables2","supp_table_6.csv"), index=False)